In [7]:
import torch
from pathlib import Path

from app.model.modules.utils.checkpointer import Checkpointer
from app.model.modules.utils.tokenizer import Tokenizer

checkpoint_path = Path('app/data/models/2/state.pkl')
model, optimizer, evaluator, config = Checkpointer.load_checkpoint(
    checkpoint_path)
tokenizer = Tokenizer()
tokenizer.load()

Loaded checkpoint app/data/models/2/state.pkl


In [8]:
sequence = 'It takes practice, Lily'
max_gen_len = 10
end_token = '<|endoftext|>'
while max_gen_len:
    tokens = torch.tensor(tokenizer.encode(sequence))
    next_token = model.generate_next_token(tokens).item()
    decoded_next_token = tokenizer.decode([int(next_token)])
    if sequence.endswith(end_token):
        break
    sequence += decoded_next_token
    max_gen_len -= 1
print(sequence)

It takes practice, Lily showed them"ond,anic stone made everyone cheeredkee
